In [1]:
def get_input(fname='test.txt'):
    input = []
    with open(fname) as f:
        for l in f.readlines():
            line = l.rstrip()
            parts = line.split(' ')
            input.append(dict(springs=list(parts[0]), sizes=[int(i) for i in parts[-1].split(',')]))
    return input

In [2]:
test_input = get_input('test.txt')
my_input = get_input('input.txt')

In [3]:
test_input

[{'springs': ['?', '?', '?', '.', '#', '#', '#'], 'sizes': [1, 1, 3]},
 {'springs': ['.',
   '?',
   '?',
   '.',
   '.',
   '?',
   '?',
   '.',
   '.',
   '.',
   '?',
   '#',
   '#',
   '.'],
  'sizes': [1, 1, 3]},
 {'springs': ['?',
   '#',
   '?',
   '#',
   '?',
   '#',
   '?',
   '#',
   '?',
   '#',
   '?',
   '#',
   '?',
   '#',
   '?'],
  'sizes': [1, 3, 1, 6]},
 {'springs': ['?', '?', '?', '?', '.', '#', '.', '.', '.', '#', '.', '.', '.'],
  'sizes': [4, 1, 1]},
 {'springs': ['?',
   '?',
   '?',
   '?',
   '.',
   '#',
   '#',
   '#',
   '#',
   '#',
   '#',
   '.',
   '.',
   '#',
   '#',
   '#',
   '#',
   '#',
   '.'],
  'sizes': [1, 6, 5]},
 {'springs': ['?', '#', '#', '#', '?', '?', '?', '?', '?', '?', '?', '?'],
  'sizes': [3, 2, 1]}]

In [4]:
from collections import deque

def arrangements(springs=[], sizes=[]):
    springs = deque(springs)
    sizes = deque(sizes)
    if len(sizes) == 0:
        return int('#' not in springs)
    if not springs:
        return set(sizes) == { 0 }
    if springs[0] == '#':
        while sizes[0] > 0 and springs:
            if springs.popleft() == '.':
                return 0
            sizes[0] -= 1
        if sizes[0] > 0:
            return 0
        if sizes[0] == 0:
            if springs and springs[0] == '#':
                return 0
            sizes.popleft()
            if len(sizes) == 0:
                return int('#' not in springs)
            if not springs:
                return 0
            springs.popleft()
            return arrangements(springs, sizes)
    # non-#, sizes > 0
    while springs and springs[0] == '.':
        springs.popleft()
    if springs and springs[0] == '?':
        springs[0] = '#'
        v1 = arrangements(springs, sizes)
        springs.popleft()
        v2 = arrangements(springs, sizes)
        return v1 + v2
    else:
        return arrangements(springs, sizes)
    return None

In [5]:
arrangements(**test_input[0])

1

In [6]:
arrangements(['#', '.', '?', '?'], [1, 1])

2

In [7]:
[arrangements(**i) for i in test_input]

[1, 4, 1, 1, 4, 10]

In [8]:
sum(arrangements(**i) for i in my_input)

6827

In [9]:
def expand_input(springs=[], sizes=[]):
    return '?'.join([''.join(springs)] * 5), tuple(sizes * 5)

In [10]:
from collections import deque
from functools import cache

@cache
def arrangements_opt(springs=[], sizes=[]):
    springs = deque(springs)
    sizes = deque(sizes)
    while springs and springs[0] == '.':
        springs.popleft()
    if not sizes:
        return '#' not in springs
    if not springs:
        return not sizes
    # Either # or '?', and have sizes
    v1 = 0
    if springs[0] == '?':
        springs[0] = '.'
        v1 = arrangements_opt(''.join(springs), tuple(sizes))
    springs[0] = '#'
    v2 = 0
    valid = len(springs) >= sizes[0]
    if valid:
        for _ in range(sizes[0]):
            if springs.popleft() == '.':
                valid = False
                break
    if valid:
        sizes.popleft()
        valid = not springs or springs[0] != '#'
    if valid:
        if springs:
            springs[0] = '.'
        v2 = arrangements_opt(''.join(springs), tuple(sizes))
    return v1 + v2

In [11]:
[arrangements_opt(springs=''.join(i['springs']), sizes=tuple(i['sizes'])) for i in test_input]

[1, 4, 1, 1, 4, 10]

In [12]:
sum(arrangements_opt(springs=''.join(i['springs']), sizes=tuple(i['sizes'])) for i in my_input)

6827

In [13]:
sum(arrangements_opt(*expand_input(**i)) for i in test_input)

525152

In [14]:
sum(arrangements_opt(*expand_input(**i)) for i in my_input)

1537505634471